# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [36]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [37]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [38]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [39]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [40]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [41]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [42]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Create all Tables




In [43]:
### Here we drop each table if it exists, and then we create the tables
drop = "DROP TABLE IF EXISTS music_app"
drop1 = "DROP TABLE IF EXISTS artist_library"
drop2 = "DROP TABLE IF EXISTS user_library"

#for the music_app table we chose as PRIMARY KEY sessionId and iteminSession, as these are the columns that we will use to make our query
create = "CREATE TABLE IF NOT EXISTS music_app(artist text, song text, length float\
                                             , sessionId int, iteminSession int\
                                             , PRIMARY KEY(sessionId, iteminSession))"

#for the artist_library table we chose as PRIMARY KEY sessionId and userId, as these are the columns that we will use to make our query and iteminSession for clustering column, because it will sort our table
create1 = "CREATE TABLE IF NOT EXISTS artist_library(artist text, song text, firstName text\
                                              , lastName text, userId int, sessionId int\
                                              , iteminSession int\
                                              , PRIMARY KEY((userId, sessionId), iteminSession))"

#for the user_library table we chose as PRIMARY KEY song and userId, as these are the columns that we will use to make our query, if we choose only song, the result the result will not be complete, because if the same userId listens to several songs, it will only return one result
create2 = "CREATE TABLE IF NOT EXISTS user_library(firstName text, lastName text\
                                                , song text, userId int, PRIMARY KEY(song, userId))"
try:
    session.execute(drop)
    session.execute(create)
    session.execute(drop1)
    session.execute(create1)
    session.execute(drop2)
    session.execute(create2)
except Exception as e:
    print(e)


                    

In [44]:
### Inserting data into tables music_app, artist_library and user_library
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app(artist, song, length, sessionId, iteminSession)"
        query = query + "values(%s, %s, %s, %s, %s)"
        
        query1 = "INSERT INTO artist_library(artist, song, firstName, lastName \
                                        , userId, sessionId, iteminSession)"
        query1 = query1 + "values(%s, %s, %s, %s, %s, %s, %s)"
        
        query2 = "INSERT INTO user_library(firstName, lastName, song, userId)"
        query2 = query2 + "values(%s, %s, %s, %s)"
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))
        session.execute(query1, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))
        session.execute(query2, (line[1], line[4], line[9], int(line[10])))

#### Query to check data entry

In [45]:
#Performing the first query on the music_app table
query = "SELECT artist, song, length FROM music_app WHERE sessionId = 338 and iteminSession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist,  row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [46]:
### The first select
#Performing the first query on the music_app table
#Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
query = "SELECT artist, song, length FROM music_app WHERE sessionId = 338 and iteminSession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)


                    

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [48]:
### The second select
#performing the first query on the artist_library table
## for userid = 10, sessionid = 182
query = "SELECT artist, song, firstname, lastname FROM artist_library WHERE sessionId = 182 and userId = 10"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)


                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [31]:
### The third select
#Performing the first query on the user_library table
query = "SELECT firstname, lastname FROM user_library WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [49]:
drop = "DROP TABLE IF EXISTS music_app"
drop1 = "DROP TABLE IF EXISTS artist_library"
drop2 = "DROP TABLE IF EXISTS user_library"
try:
    session.execute(drop)
    session.execute(drop1)
    session.execute(drop2)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [50]:
session.shutdown()
cluster.shutdown()